#### MP based asynchronous testing

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import multiprocessing as mp
import TestPool_Unit
from shutil import copyfile
import numpy as np

In [3]:
from collections import defaultdict

In [4]:
def Evaluate(result_arr):
    print('Files Processed: ', len(result_arr))
    recalls = []
    recalls_of_word = []
    precisions = []
    precisions_of_words = []
    fully_Correct_l = 0
    fully_Correct_w = 0
    for entry in result_arr:
        (word_match, lemma_match, n_dcsWords, n_output_nodes) = entry
        recalls.append(lemma_match/n_dcsWords)
        recalls_of_word.append(word_match/n_dcsWords)

        precisions.append(lemma_match/n_output_nodes)
        precisions_of_words.append(word_match/n_output_nodes)
        if lemma_match == n_dcsWords:
            fully_Correct_l += 1
        if word_match == n_dcsWords:
            fully_Correct_w += 1
    print('Avg. Micro Recall of Lemmas: {}'.format(np.mean(np.array(recalls))))
    print('Avg. Micro Recall of Words: {}'.format(np.mean(np.array(recalls_of_word))))
    print('Avg. Micro Precision of Lemmas: {}'.format(np.mean(np.array(precisions))))
    print('Avg. Micro Precision of Words: {}'.format(np.mean(np.array(precisions_of_words))))
    print('Fully Correct Lemmawise: {}'.format(fully_Correct_l/len(recalls_of_word)))
    print('Fully Correct Wordwise: {}'.format(fully_Correct_w/len(recalls_of_word)))

In [5]:
modelFile = 'outputs/train_t2009110446493/nnet_i50.p' #se_loss Rerun
_dump = False
if _dump:
    _outFile = 'outputs/dump_predictions/pred_1L_bigram'
else:
    _outFile = None

# Backup the model file
copyfile(modelFile, modelFile + '.bk')

# Create Queue, Result array
queue = mp.Queue()
result_arr = []

# Start 6 workers - 8 slows down the pc
proc_count = 4
procs = [None]*proc_count
for i in range(proc_count):
    vpid = i
    procs[i] = mp.Process(target = TestPool_Unit.pooled_Test, args = \
                          (modelFile, vpid, queue, '../NewData/skt_dcs_DS.bz2_1L_bigram_heldout/', 2400, _dump, _outFile))
# Start Processes
for i in range(proc_count):
    procs[i].start()

Child process with vpid:0, pid:29324 started.
Child process with vpid:1, pid:29325 started.
Child process with vpid:2, pid:29328 started.
Child process with vpid:3, pid:29333 started.
vpid:0: Range is 0 -> 2400 / 9577
vpid:2: Range is 4800 -> 7200 / 9577
vpid:1: Range is 2400 -> 4800 / 9577
vpid:3: Range is 7200 -> 9600 / 9577


In [7]:
# Fetch partial results
while not queue.empty():
    result_arr.append(queue.get())
# Evaluate results till now
Evaluate(result_arr)

Files Processed:  1209
Avg. Micro Recall of Lemmas: 0.8449972847287107
Avg. Micro Recall of Words: 0.7388494372146394
Avg. Micro Precision of Lemmas: 0.7921270283574317
Avg. Micro Precision of Words: 0.693247785837477
Fully Correct Lemmawise: 0.3771712158808933
Fully Correct Wordwise: 0.19106699751861042


In [11]:
# Check status
for i in range(proc_count):
    p = procs[i]
    print('Process with\t vpid: {}\t ->\t pid: {}\t ->\t running status: {}'.format(i, p.pid, p.is_alive()))

Process with	 vpid: 0	 ->	 pid: 4770	 ->	 running status: False
Process with	 vpid: 1	 ->	 pid: 4771	 ->	 running status: False
Process with	 vpid: 2	 ->	 pid: 4772	 ->	 running status: False
Process with	 vpid: 3	 ->	 pid: 4777	 ->	 running status: False


In [17]:
# Properly Join
for i in range(proc_count):
    procs[i].join()

In [10]:
# Force Terminate
for p in procs:
    p.terminate()

In [24]:
np.mean([a[2] for a in result_arr])

6.7729026036644164

In [17]:
result_arr2 = [list(x) for x in result_arr]

recalls = defaultdict(list)
for i in range(len(result_arr2)):
    result_arr2[i][0] /= result_arr2[i][2]
    result_arr2[i][1] /= result_arr2[i][2]
    recalls[result_arr2[i][2]].append(result_arr2[i][1])

for c in sorted(recalls.keys()):
    print('C: {}, Samples: {}, Mean: {}'.format(c, len(recalls[c]), np.mean(recalls[c])))

C: 1, Samples: 110, Mean: 0.7545454545454545
C: 2, Samples: 245, Mean: 0.8244897959183674
C: 3, Samples: 167, Mean: 0.8622754491017964
C: 4, Samples: 142, Mean: 0.8151408450704225
C: 5, Samples: 367, Mean: 0.8288828337874659
C: 6, Samples: 1114, Mean: 0.8708856971873129
C: 7, Samples: 1500, Mean: 0.8959047619047618
C: 8, Samples: 1066, Mean: 0.8903611632270169
C: 9, Samples: 401, Mean: 0.888057633693544
C: 10, Samples: 122, Mean: 0.8811475409836067
C: 11, Samples: 57, Mean: 0.9027113237639554
C: 12, Samples: 42, Mean: 0.8432539682539681
C: 13, Samples: 18, Mean: 0.8717948717948718
C: 14, Samples: 6, Mean: 0.8571428571428571
C: 15, Samples: 10, Mean: 0.8466666666666667
C: 16, Samples: 4, Mean: 0.890625
C: 17, Samples: 9, Mean: 0.8431372549019608
C: 18, Samples: 6, Mean: 0.8888888888888888
C: 19, Samples: 1, Mean: 0.8421052631578947
C: 21, Samples: 4, Mean: 0.9166666666666666
C: 22, Samples: 2, Mean: 0.8181818181818181
C: 23, Samples: 2, Mean: 0.8913043478260869
C: 24, Samples: 1, Mean: 